In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
### 週販レポート2025年版

# 標準ライブラリ
from datetime import datetime, timedelta

# サードパーティライブラリ
import polars as pl
import matplotlib.pyplot as plt
import japanize_matplotlib
from matplotlib.ticker import ScalarFormatter
from IPython.display import Markdown, display

import gamedata as g

pl.Config.set_tbl_rows(40)
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [5]:
event_df = g.load_hard_event()
event_df

event_date,hw,event_name,event_type,priority,report_date
date,str,str,str,f64,date
1999-02-11,"""PS""","""FF8""","""soft""",0.20,1999-02-14
1999-11-21,"""GB""","""ポケモン金銀""","""soft""",2.00,1999-11-21
1999-12-11,"""PS""","""GT2""","""soft""",0.30,1999-12-12
1999-12-29,"""DC""","""シェンムー""","""soft""",0.40,2000-01-02
2000-07-07,"""PS""","""FF9""","""soft""",1.50,2000-07-09
2000-08-26,"""PS""","""DQ7""","""soft""",1.50,2000-08-27
2001-01-31,"""DC""","""DC撤退発表""","""hard""",0.50,2001-02-04
2001-04-28,"""PS2""","""GT3""","""soft""",1.00,2001-04-29
2001-07-19,"""PS2""","""FFX""","""soft""",0.20,2001-07-22


In [8]:
info_df = g.load_hard_info()
delta_event_df = g.delta_event(event_df, info_df)
delta_event_df

report_date,event_date,hw,event_name,event_type,priority,delta_week
date,date,str,str,str,f64,i32
1999-02-14,1999-02-11,"""PS""","""FF8""","""soft""",0.20,219
1999-11-21,1999-11-21,"""GB""","""ポケモン金銀""","""soft""",2.00,552
1999-12-12,1999-12-11,"""PS""","""GT2""","""soft""",0.30,262
2000-01-02,1999-12-29,"""DC""","""シェンムー""","""soft""",0.40,57
2000-07-09,2000-07-07,"""PS""","""FF9""","""soft""",1.50,292
2000-08-27,2000-08-26,"""PS""","""DQ7""","""soft""",1.50,299
2001-02-04,2001-01-31,"""DC""","""DC撤退発表""","""hard""",0.50,114
2001-04-29,2001-04-28,"""PS2""","""GT3""","""soft""",1.00,60
2001-07-22,2001-07-19,"""PS2""","""FFX""","""soft""",0.20,72


In [9]:
masked_event_df = g.mask_event(delta_event_df, g.EVENT_MASK_LONG)
masked_event_df


report_date,event_date,hw,event_name,event_type,priority,delta_week
date,date,str,str,str,f64,i32
2001-02-04,2001-01-31,"""DC""","""DC撤退発表""","""hard""",0.50,114
2004-12-05,2004-12-02,"""GBA""","""DS発売""","""hard""",0.50,193
2007-09-23,2007-09-20,"""PSP""","""PSP-2000""","""hard""",0.30,145
2008-11-02,2008-11-01,"""DS""","""DSi発売""","""hard""",0.30,204
2009-09-06,2009-09-03,"""PS3""","""PS3 Slim""","""hard""",0.10,147
2010-06-20,2010-06-15,"""DS""","""3DS発表""","""hard""",0.50,289
2011-07-31,2011-07-28,"""3DS""","""3DS値下げ発表""","""hard""",0.30,22
2011-08-14,2011-08-11,"""3DS""","""3DS新価格""","""hard""",0.20,24
2012-07-29,2012-07-28,"""3DS""","""3DSLL""","""hard""",0.50,74


In [11]:
filtered_event_df = g.filter_event(delta_event_df, start_date=datetime(2024,1,1), 
                                   hw=['NSW', 'NS2'], event_mask=g.EVENT_MASK_MIDDLE)
filtered_event_df

report_date,event_date,hw,event_name,event_type,priority,delta_week
date,date,str,str,str,f64,i32
2024-10-20,2024-10-17,"""NSW""","""マリパジャンボリー""","""soft""",1.50,398
2025-04-06,2025-04-02,"""NSW""","""Switch2 Direct""","""hard""",0.50,422
2025-06-08,2025-06-05,"""NSW""","""Switch2発売""","""hard""",1.00,431
2025-07-20,2025-07-17,"""NS2""","""DKバナンザ""","""soft""",0.50,6
2025-10-19,2025-10-16,"""NS2""","""ポケモンZA""","""soft""",0.50,19
2025-11-23,2025-11-20,"""NS2""","""エアライダー""","""soft""",1.00,24
2025-12-07,2025-12-04,"""NS2""","""Metroid Prime 4""","""soft""",0.50,26
2026-01-25,2026-01-22,"""NS2""","""FF7R""","""soft""",1.50,33
2026-02-08,2026-02-05,"""NS2""","""DQ7R""","""soft""",1.50,35


In [13]:
base_df = g.date_filter(g.load_hard_sales(), begin=datetime(2023,1,1))
pivot_df = g.pivot_sales(base_df)
event_pos_df = g.add_event_positions(event_df, pivot_df, event_mask=g.EVENT_MASK_SHORT)
event_pos_df

event_date,hw,event_name,event_type,priority,report_date,x_pos,y_pos
date,str,str,str,f64,date,date,i64
2023-02-22,"""PS5""","""PSVR2""","""hard""",1.80,2023-02-26,2023-02-22,"91,729"
2023-05-12,"""NSW""","""ToTK""","""soft""",2.00,2023-05-14,2023-05-12,"83,052"
2023-06-22,"""PS5""","""FF16""","""soft""",1.50,2023-06-25,2023-06-22,"85,385"
2023-07-21,"""NSW""","""ピクミン4""","""soft""",3.00,2023-07-23,2023-07-21,"71,180"
2023-08-01,"""PS5""","""PS5値上げ2""","""price""",0.00,2023-08-06,2023-08-01,"50,358"
2023-10-20,"""NSW""","""マリオワンダー""","""soft""",3.00,2023-10-22,2023-10-20,"62,250"
2023-11-10,"""PS5""","""PS5 Slim""","""hard""",1.00,2023-11-12,2023-11-10,"107,953"
2024-02-29,"""PS5""","""FF7R2""","""soft""",0.40,2024-03-03,2024-02-29,"48,067"
2024-08-27,"""PS5""","""PS5値上げ3""","""price""",0.00,2024-09-01,2024-08-27,"33,865"


In [16]:
pivot_delta_df = g.pivot_sales_by_delta(base_df)
event_pos_delta_df = g.add_event_positions_delta(delta_event_df, pivot_delta_df, event_mask=g.EVENT_MASK_SHORT)
event_pos_delta_df

report_date,event_date,hw,event_name,event_type,priority,delta_week,x_pos,y_pos
date,date,str,str,str,f64,i32,i64,i64
2023-02-26,2023-02-22,"""PS5""","""PSVR2""","""hard""",1.80,119,119,"91,729"
2023-05-14,2023-05-12,"""NSW""","""ToTK""","""soft""",2.00,323,323,"83,052"
2023-06-25,2023-06-22,"""PS5""","""FF16""","""soft""",1.50,136,136,"85,385"
2023-07-23,2023-07-21,"""NSW""","""ピクミン4""","""soft""",3.00,333,333,"71,180"
2023-08-06,2023-08-01,"""PS5""","""PS5値上げ2""","""price""",0.00,142,142,"50,358"
2023-10-22,2023-10-20,"""NSW""","""マリオワンダー""","""soft""",3.00,346,346,"62,250"
2023-11-12,2023-11-10,"""PS5""","""PS5 Slim""","""hard""",1.00,156,156,"107,953"
2024-03-03,2024-02-29,"""PS5""","""FF7R2""","""soft""",0.40,172,172,"48,067"
2024-09-01,2024-08-27,"""PS5""","""PS5値上げ3""","""price""",0.00,198,198,"33,865"
